Crawler to get info from Fotocasa

In [1]:
!pip install selenium

     |████████████████████████████████| 904 kB 4.6 MB/s eta 0:00:01


In [64]:
import pandas as pd
import numpy
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

In [3]:
#driver=webdriver.Chrome('/Users/pablogoya/Documents/Master DS/chromedriver')

!pip install webdriver-manager
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType

driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.GOOGLE).install())

driver.get("https://www.fotocasa.es/es/")
driver.maximize_window()

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - There is no [mac64] chromedriver for browser 87.0.4280 in cache
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/pablogoya/.wdm/drivers/chromedriver/mac64/87.0.4280.88]


In [5]:
time.sleep(2)
buy=driver.find_element_by_xpath('.//div[@class="re-Search-selectorContainer re-Search-selectorContainer--buy"]')
buy.click()

In [6]:
time.sleep(1)
buscador=driver.find_element_by_xpath('.//div[@class="sui-MoleculeAutosuggest-input-container"]/input')
buscador.click()
buscador.send_keys('Madrid')
time.sleep(1)
buscador.send_keys(Keys.ENTER)

In [120]:
#Scrapping the FotoCasa website with multiple for loops

def ftcs_scrapper(x):
    
    #Cleaning lists:
    
    list_prices=[]
    list_features=[]
    list_house_types=[]
    list_locations=[]
    list_prices_full=[]
    list_features_full=[]
    list_house_types_full=[]
    list_locations_full=[]

    
    for u in range (1,x+2):

        url='https://www.fotocasa.es/es/comprar/viviendas/madrid-capital/todas-las-zonas/l/'
        parameters='?conservationStatusIds=1;3;8;4;2'
        driver.get(url+str(u)+parameters)
        list_prices_full=list_prices_full+list_prices
        list_features_full=list_features_full+list_features
        list_house_types_full=list_house_types_full+list_house_types
        list_locations_full=list_locations_full+list_locations

        for i in range (15):

            html_txt=driver.page_source
            soup=BeautifulSoup(html_txt)

            prices=soup.find_all('div',class_="re-Card-priceComposite")
            features=soup.find_all('span',class_="re-CardFeatures-feature")  
            house_types=soup.find_all('div' ,class_="re-Card-wrapperTitle")
            locations=soup.find_all('h3' ,class_="re-Card-title")

            list_prices=[]
            list_features=[]
            list_house_types=[]
            list_locations=[]

            for price in prices:
                title1=price.find('span').getText()
                list_prices.append(title1)
            for feature in features:
                title2=feature.getText()
                list_features.append(title2)
            for house_type in house_types:
                title3=house_type.getText()
                list_house_types.append(title3)
            for location in locations:
                title4=location.getText()
                list_locations.append(title4)

            ActionChains(driver).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).perform()
            time.sleep(0.5)
    
    
    #Building the full DataFrame with all read data through the scrapper

    houses_list_full=[list_prices_full,list_locations_full]
    #list_features_full_splitted = [list_features_full[i:i + 4] for i in range(0, len(list_features_full),4)]
    list_features_full_splitted = [list_features_full[i*4:(i+1)*4] for i in range((len(list_features_full)+4-1)//4)] 
    
    features_df=pd.DataFrame(list_features_full_splitted)
    features_df.columns=['rooms','bathrooms','size','carasteristics']
    
    result_ftcs_df=pd.DataFrame (houses_list_full).transpose()
    result_ftcs_df.columns=['prices','locations']
    result_ftcs_df= pd.concat([result_ftcs_df, features_df], axis=1)            
            

    return display(result_ftcs_df)


In [170]:
#Cleaning lists:

list_prices=[]
list_features=[]
list_house_types=[]
list_locations=[]
list_prices_full=[]
list_features_full=[]
list_house_types_full=[]
list_locations_full=[]


for u in range (1,4):

    url='https://www.fotocasa.es/es/comprar/viviendas/madrid-capital/todas-las-zonas/l/'
    parameters='?conservationStatusIds=1;3;8;4;2'
    driver.get(url+str(u)+parameters)
    list_prices_full=list_prices_full+list_prices
    list_features_full=list_features_full+list_features
    list_house_types_full=list_house_types_full+list_house_types
    list_locations_full=list_locations_full+list_locations

    for i in range (15):

        html_txt=driver.page_source
        soup=BeautifulSoup(html_txt)

        prices=soup.find_all('div',class_="re-Card-priceComposite")
        features=soup.find_all('span',class_="re-CardFeatures-feature")  
        house_types=soup.find_all('div' ,class_="re-Card-wrapperTitle")
        locations=soup.find_all('h3' ,class_="re-Card-title")

        list_prices=[]
        list_features=[]
        list_house_types=[]
        list_locations=[]

        for price in prices:
            title1=price.find('span').getText()
            list_prices.append(title1)
        for feature in features:
            title2=feature.getText()
            list_features.append(title2)
        for house_type in house_types:
            title3=house_type.getText()
            list_house_types.append(title3)
        for location in locations:
            title4=location.getText()
            list_locations.append(title4)

        ActionChains(driver).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).perform()
        time.sleep(0.5)

In [119]:
ftcs_scrapper(2)

,prices,locations,rooms,bathrooms,size,carasteristics
0,280.000 €,PisoChamartín,1 hab.,1 baño,40 m²,
1,385.000 €,"ApartamentoCalle de la Esperanza, Centro",1 hab.,1 baño,71 m²,con ascensor y terraza
2,850.000 €,DúplexFuencarral,4 habs.,4 baños,295 m²,con ascensor
3,234.000 €,PisoFuencarral,2 habs.,1 baño,65 m²,
4,335.000 €,"PisoCalle de Jaime el Conquistador, Arganzuela",4 habs.,1 baño,100 m²,con ascensor
...,...,...,...,...,...,...
62,1.019.900 €,"Planta bajaCalle Ferraz, 41, Moncloa",2 baños,165 m²,con ascensor y parking,1 hab.
63,370.000 €,"Planta bajaCalle Cerro del Campo, 42, Vicálvaro",1 baño,55 m²,con ascensor,4 habs.
64,199.900 €,"PisoAtocha, Centro",3 baños,264 m²,con parking y terraza,5 habs.
65,487.600 €,"Casa o chaletCalle Igualdad, 171, Vicálvaro",2 baños,176 m²,con ascensor,None


In [171]:
#Building the full DataFrame with all read data through the scrapper

houses_list_full=[list_prices_full,list_locations_full]
#list_features_full_splitted = [list_features_full[i:i + 4] for i in range(0, len(list_features_full),4)]
list_features_full_splitted = [list_features_full[i*4:(i+1)*4] for i in range((len(list_features_full)+4-1)//4)] 

features_df=pd.DataFrame(list_features_full_splitted)
features_df.columns=['rooms','bathrooms','size','carasteristics']

result_ftcs_df=pd.DataFrame (houses_list_full).transpose()
result_ftcs_df.columns=['prices','locations']
result_ftcs_df= pd.concat([result_ftcs_df, features_df], axis=1)  

In [136]:
pd.set_option('display.max_rows', None)
features_df

,rooms,bathrooms,size,carasteristics
0,5 habs.,5 baños,416 m²,con terraza
1,2 habs.,2 baños,145 m²,con ascensor
2,4 habs.,4 baños,450 m²,con terraza
3,4 habs.,4 baños,295 m²,con ascensor
4,5 habs.,6 baños,314 m²,con ascensor y terraza
5,8 habs.,5 baños,1060 m²,con terraza
6,3 habs.,1 baño,72 m²,con terraza
7,4 habs.,4 baños,340 m²,
8,2 habs.,2 baños,109 m²,con ascensor y terraza
9,2 habs.,1 baño,66 m²,


In [172]:
rooms=[k for k in list_features_full if 'hab' in k]
baths=[k for k in list_features_full if 'ba' in k]
sizes=[k for k in list_features_full if 'm²' in k]

In [173]:
#Building the full DataFrame with all read data through the scrapper

houses_list_full=[list_prices_full,list_locations_full,rooms, baths, sizes,] 

result_ftcs_df=pd.DataFrame (houses_list_full).transpose()
result_ftcs_df.columns=['prices','locations','rooms','bathrooms','size']


In [178]:
result_ftcs_df.loc[result_ftcs_df['locations'].str.contains('Hortaleza','Chamberí'),['locations']]='Arganzuela'

In [179]:
result_ftcs_df

,prices,locations,rooms,bathrooms,size
0,630.000 €,PisoChamartín,3 habs.,2 baños,112 m²
1,1.586.000 €,"Casa adosadaCalle de José Rizal, Hortaleza",5 habs.,5 baños,416 m²
2,280.000 €,ApartamentoSalamanca,1 hab.,1 baño,39 m²
3,940.000 €,PisoChamartín,4 habs.,3 baños,210 m²
4,3.100.000 €,"Casa o chaletAvenida de Los Madroños, Hortaleza",6 habs.,5 baños,750 m²
5,1.590.000 €,DúplexCiudad Lineal,3 habs.,3 baños,244 m²
6,650.000 €,PisoSalamanca,4 habs.,4 baños,163 m²
7,1.950.000 €,PisoCentro,6 habs.,4 baños,455 m²
8,280.000 €,"PisoCalle María Lejárraga, Carabanchel",2 habs.,2 baños,109 m²
9,163.000 €,"PisoCalle de las Madres de la Plaza de Mayo, C...",2 habs.,1 baño,66 m²
